<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/4_03_DETECTRON2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectron2 
Detectron2 to system detekcji obiektów stworzony przez Facebook AI Research, posiadający implementacje najnowocześniejszych algorytmów wykrywania obiektów w tym Faster R-CNN, Mask R-CNN (state-of-the-art object detection algorithms). System został napisany w PyTorchu. Przez twórców został udostępniony jako biblioteka open-source, która może być wykorzystana do obsługi i rozwoju różnych projektów badawczych.




### Instalacja środowiska

Środowisko instalowane jest z repozytorium umieszczonym przez Facebook AI Research na [GitHub](https://github.com/facebookresearch/detectron2)

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -q -e detectron2_repo

Po instalacji detectron2 w środowisku Colab niezbędne jest ponowne uruchomienie środwiska wykonawczego (Ctrl+M.)

Import podstawowych modułów:

In [ ]:
from detectron2 import model_zoo
import detectron2.utils.comm as comm
from detectron2.engine import DefaultPredictor, DefaultTrainer, HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader
from detectron2.structures import BoxMode
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

### Struktura repozytorium
Poniżej znajduje się drzewo katalogów detectron2 i opis zawartości katalogów.

```
detectron2/

├─checkpoint  <- checkpointer 
├─config      <- domyślne konfiguracje
├─data        <- programy obsługi zbiorów danych i programy ładujące dane
├─engine      <- moduły trenowania i predykcji
├─evaluation  <- evaluatory dla podstawowych typów zbiorów danych
├─export      <- konwerter modeli detectron2 do caffe2 (ONNX)
├─layers      <- warstwy niestandardowe np.: Deformable convolution
├─model_zoo   <- wytreniwane modele
├─modeling   
│  ├─meta_arch <- meta architektury np. R-CNN, RetinaNet
│  ├─backbone  
│  │    ├─backbone.py   <- zawiera abstrakcyjną klasę bazową Backbone
│  │    ├─build.py      <- wywołanie funkcji konstruktora określonej w config
│  │    ├─fpn.py        <- zawiera klasę FPN 
│  │    ├─resnet.py     <- zawiera klasę ResNet 
│  ├─proposal_generator <- RPN
│  └─roi_heads <- ROI heads 
├─solver       <- moduły zawierające modyfikacje optimizera i schedulera
├─structures   <- klasy struktur np.: Boxes, Instances
└─utils        <- moduły użytkowe np.: visualizer, logger 	
```

### Omówienie detectron2


Architektura detectron2 zostanie omówiona na przykładzie Faster R-CNN z FPN Resnet50. Podstawową konfiguracją Faster R-CNN z FPN Resnet50 jest [Base-RCNN-FPN](https://github.com/facebookresearch/detectron2/blob/master/configs/Base-RCNN-FPN.yaml).
Base-RCNN-FPN określa przede wszystkim meta architekturę GeneralizedRCNN, Backbone Network, RPN i ROI Heads. Klasy realizujące tą architekturę zostały zapisane w poniższych lokalizacjach:

GeneralizedRCNN [meta_arch/rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/meta_arch/rcnn.py)
 

- Backbone Network: FPN (Feature Pyramid Network) [backbone/fpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/backbone/fpn.py)
  - ResNet [backbone/resnet.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/backbone/resnet.py)

- RPN (Region Proposal Network) [proposal_generator/rpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn.py)
  - StandardRPNHead [proposal_generator/rpn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn.py)
  - RPNOutput [proposal_generator/rpn_outputs.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/proposal_generator/rpn_outputs.py)
 
- ROI Heads: StandardROIHeads [roi_heads/roi_heads.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/roi_heads.py)
 - ROIPooler [poolers.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/poolers.py)
 - FastRCNNConvFCHead [roi_heads/box_heads.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/box_head.py)
 - FastRCNNOutputLayers [roi_heads/fast_rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/fast_rcnn.py)
 - FastRCNNOutputs [roi_heads/fast_rcnn.py](https://github.com/facebookresearch/detectron2/blob/master/detectron2/modeling/roi_heads/fast_rcnn.py)

<img src="https://raw.githubusercontent.com/DarekGit/FACES_DNN/master/Figures/detectron2.png" alt="detectron2">

## Backbone Network

Implementację Feature Pyramid Network zawiera klasa FPN zapisana w pliku **backbone/fpn.py**. Wybór i konfiguracja sieci Resnet50 realizowana jest poprzez wywołanie funkcji **build_resnet_fpn_backbone** przyjmującej jako jeden z argumentów słownik **cfg** zawierający konfigurację całej architektury w tym głębokość modelu. Wybierając model Resnet50 otrzymujemy poniższa strukturę wynikową:

```
ResNet(
  (stem): BasicStem()              <- BasicStem blok 
  (res2):                          <- blok res2
    (0): BottleneckBlock()         <- (shortcut), stride=(1, 1)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
  (res3):                          <- blok res3
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1) 
    (2): BottleneckBlock()         <- stride=(1, 1)
    (3): BottleneckBlock()         <- stride=(1, 1)
  (res4):                          <- blok res4
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
    (3): BottleneckBlock()         <- stride=(1, 1)
    (4): BottleneckBlock()         <- stride=(1, 1)
    (5): BottleneckBlock()         <- stride=(1, 1)
  (res5):                          <- blok res5
    (0): BottleneckBlock()         <- (shortcut), stride=(2, 2)
    (1): BottleneckBlock()         <- stride=(1, 1)
    (2): BottleneckBlock()         <- stride=(1, 1)
)
```
Struktura wszystkich bloków modelu została zapisana w pliku [faster_rcnn_R_50_FPN.txt](faster_rcnn_R_50_FPN.txt) <br>
Reprezentuje ona model wygenerowany z konfiguracji [COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml](https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml).

- **BasicStem blok** jest realizowany przez klasę [BasicStem](https://github.com/facebookresearch/detectron2/blob/5b8f68436651c74608a3f4909cae570dbe23e51d/detectron2/modeling/backbone/resnet.py#L331). Składa się z konwolucji z kernel_size=(7, 7), stride=(2, 2), batch normalizacji. Po konwolucji wywołana jest funkcja aktywacji relu i max pooling kernel_size=3 i stride=2. Na wyjściu otrzymujemy tensor którego W, H są czterokrotnie mniejsze od rozmiaru obrazu na wejściu. 

- **BottleneckBlock** zastosowano aby zwiększyć głębokość sieci przy jednoczesnym zmniejszeniu parametrów. BottleneckBlock składa się z trzech konwolucji z kernel_size kolejno: (1, 1), (3, 3), (1, 1) ze stride=(1, 1). 
  
  Występują tu trzy tyby bloków BottleneckBlock:

  - BottleneckBlock z shortcut i stride=(1, 1)
```
      (res2): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv1): Conv2d(
            64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
```
  - BottleneckBlock z shortcut i stride=(2, 2)
```
      (res4): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
          (conv1): Conv2d(
            512, 256, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv2): Conv2d(
            256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv3): Conv2d(
            256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
        )
```
  - BottleneckBlock z stride=(1, 1) bez połączenie skrótowego
```
      BottleneckBlock(
          (conv1): Conv2d(
            512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=128, eps=1e-05)
          )
          (conv2): Conv2d(
            128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=128, eps=1e-05)
          )
          (conv3): Conv2d(
            128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
        )
```




Region Proposal Network

ROI Heads

Aby załadować dane z datasetu niezbędne jest zarejestrowanie go z wykorzystaniem metody register klasy DatasetCatalog. 

```
from detectron2.data import DatasetCatalog, MetadataCatalog

train = annotations("train")
val = annotations('val')

for d in ["train","val"]:
  DatasetCatalog.register("face_" + d, lambda d=d: train if d == "train" else val)
  MetadataCatalog.get("face_" + d).set(thing_classes = ['face'])

faces_metadata = MetadataCatalog.get("face_train")

```